In this notebook we will a model to detect potholes using the imported data. We will create different models and log results of the run in MLflow experiment.

In [0]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
from itertools import cycle
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import lasso_path, enet_path
from sklearn import datasets
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split, StratifiedKFold,StratifiedShuffleSplit
from pyspark.sql.functions import countDistinct
from sklearn.preprocessing import StandardScaler
import collections
from collections import namedtuple

from sklearn import svm
from sklearn.svm import SVC
from sklearn import metrics
from sklearn import tree
from sklearn.naive_bayes     import MultinomialNB
from sklearn.ensemble        import RandomForestClassifier
from sklearn.linear_model    import LogisticRegression
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import VarianceThreshold

import operator
import itertools
from collections import namedtuple
import warnings
warnings.filterwarnings('ignore')


from sklearn.model_selection import cross_val_score

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report, precision_recall_fscore_support

from pyspark.sql.types import StructField
from pyspark.sql.types import StructType

/databricks/python/lib/python3.5/site-packages/mlflow/utils/environment.py:26: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
 env = yaml.load(_conda_header)

In [0]:
original_data = spark.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/aggregated_data.csv')
main_cols= ['accelX(g)_mean', 'accelX(g)_std', 'accelX(g)_min', 'accelX(g)_max',
            'accelY(g)_mean', 'accelY(g)_std', 'accelY(g)_min', 'accelY(g)_max',
            'accelZ(g)_mean', 'accelZ(g)_std', 'accelZ(g)_min', 'accelZ(g)_max',
            'gyroX(rad/s)_mean', 'gyroX(rad/s)_std', 'gyroX(rad/s)_min',
            'gyroX(rad/s)_max', 'gyroY(rad/s)_mean', 'gyroY(rad/s)_std',
            'gyroY(rad/s)_min', 'gyroY(rad/s)_max', 'Speed(m/s)_min', 'Speed(m/s)_max',
            'Speed(m/s)_mean', 'Speed(m/s)_std', 
             'energy_y', 'energy_x', 'energy_z',
             'entropy_y', 'entropy_x', 'entropy_z', 'accel_mag_mean','accel_mag_std', 'accel_mag_min', 
            'accel_mag_max','gyro_mag_mean','gyro_mag_std', 'gyro_mag_min','gyro_mag_max','label']
sensor_data_org=original_data[main_cols].toPandas()

# display(sensor_data[[main_cols]])

In [0]:
display(sensor_data_org)

accelX(g)_mean,accelX(g)_std,accelX(g)_min,accelX(g)_max,accelY(g)_mean,accelY(g)_std,accelY(g)_min,accelY(g)_max,accelZ(g)_mean,accelZ(g)_std,accelZ(g)_min,accelZ(g)_max,gyroX(rad/s)_mean,gyroX(rad/s)_std,gyroX(rad/s)_min,gyroX(rad/s)_max,gyroY(rad/s)_mean,gyroY(rad/s)_std,gyroY(rad/s)_min,gyroY(rad/s)_max,Speed(m/s)_min,Speed(m/s)_max,Speed(m/s)_mean,Speed(m/s)_std,energy_y,energy_x,energy_z,entropy_y,entropy_x,entropy_z,accel_mag_mean,accel_mag_std,accel_mag_min,accel_mag_max,gyro_mag_mean,gyro_mag_std,gyro_mag_min,gyro_mag_max,label
0.044386,0.049160687586729285,-0.0505,0.1841,-1.002258,0.05156545777940889,-1.1004,-0.8955,0.04257400000000002,0.13777823312845902,-0.2188,0.4283,-0.0034820000000000025,0.1044407270943668,-0.3445,0.1761,0.0068540000000000025,0.10088140008941192,-0.2103,0.2624,8.1356,10.9444,9.708528,1.3942516534743643,2.3123175546043795,0.21934451000000008,0.3930856393643379,1241.8714889490573,-0.034653544855847235,-3.410286790811824,1.0147950619421175,0.05053072785834122,0.8966624169663853,1.1052945354067396,0.12753446963560325,0.07732010382019279,0.011825819210524065,0.3851270050256149,0.0
0.039458,0.09693705192546348,-0.2887,0.2435,-1.0059360000000002,0.0948745904022779,-1.2087,-0.6925,0.09138200000000002,0.12318244305094785,-0.2169,0.4283,0.027926,0.13594737924653053,-0.4026,0.3914,0.013410000000000005,0.14483338461832618,-0.2103,0.3563,10.9444,13.0293,11.277984000000002,0.7643369653130743,2.3129755603355475,0.5476862900000001,0.6614771404073017,415.1737491528443,-0.08625494242224731,-2.8986892011537786,1.0229686643589553,0.0944555225471506,0.7109048459533808,1.2237092383405463,0.18233137716475328,0.0986729866843225,0.03226794074619575,0.4697770109317824,0.0
0.040458,0.0916418094321582,-0.2887,0.2435,-1.0007139999999999,0.11848081702959345,-1.2408,-0.6925,0.038096,0.10960719494631728,-0.1908,0.2878,0.011146000000000005,0.20583817450609107,-0.4656,0.3914,-0.006396000000000002,0.12180736916952108,-0.2093,0.3563,10.9444,13.6263,12.684144000000002,1.0083489465775228,2.4150866871474914,0.50175355,0.4616270904400628,455.52691440011546,-0.07932865600803547,-3.5111005006734928,1.01230538534061,0.11918109753815385,0.7109048459533808,1.2452231285998503,0.21370080527682056,0.1234223214173126,0.02692303846151099,0.4848861412744233,0.0
0.025884,0.048545683062451606,-0.0826,0.1679,-1.0059539999999998,0.09633891884384004,-1.2408,-0.8176,0.041405999999999984,0.07606791809955099,-0.1346,0.237,0.013336,0.18546761200813472,-0.4656,0.3511,-0.005902000000000001,0.07073280565621584,-0.1473,0.1668,13.0293,13.6263,13.394076,0.2394668691573016,1.4730797278374328,0.15133324,0.2082985719093237,1938.3348614562494,-0.026955996407683296,-3.1908909767921783,1.0111551984737472,0.0965259125805974,0.8182318803371084,1.2452231285998503,0.17264700372486494,0.11018884383106337,0.034198976592874825,0.4848861412744233,0.0
0.025818,0.0488172047130927,-0.0842,0.1379,-1.002008,0.07551411746157137,-1.1711,-0.8252,0.049616,0.05554359858705592,-0.0773,0.1634,0.017588000000000003,0.12948664431515708,-0.2521,0.3169,0.0020780000000000013,0.07165290863600722,-0.1671,0.2011,13.2134,13.6263,13.229916000000001,0.08091154518361403,1.4428265348581844,0.15248443,0.17173276680506708,1903.7374848401964,-0.027087148434824158,-2.5795637738317985,1.006222064410143,0.07639436166491828,0.8297453344249668,1.1725857836422886,0.1294895939045594,0.08203170283757552,0.019377306314346168,0.34084361516683864,0.0
0.035246,0.04960576260879375,-0.0842,0.1192,-1.015798,0.08327600012008261,-1.1789,-0.8252,0.04566,0.04682399384930765,-0.054000000000000006,0.1414,0.034558,0.13389810915767258,-0.2371,0.4115,0.0070160000000000005,0.07527115877944221,-0.1671,0.2011,13.2134,13.4808,13.229444000000004,0.06350398148147886,1.9479797144297004,0.18515061,0.1669690926860569,1564.207535029717,-0.029866238364494275,-2.4554385058881603,1.0196594419276683,0.08399658140517305,0.8297453344249668,1.1857365896353205,0.1373506082187248,0.08664643109757245,0.0228372502723073,0.428912147181681,0.0
0.01

In [0]:
def variance_threshold_selector(data, threshold=0.5):
    selector = VarianceThreshold(threshold)
    selector.fit(data)
    return data[data.columns[selector.get_support(indices=True)]]

In [0]:
main_columns= variance_threshold_selector(sensor_data, 0.16).columns.tolist()

In [0]:
main_columns

Out[ 25 ]: 
['Speed(m/s)_min',
 'Speed(m/s)_max',
 'Speed(m/s)_mean',
 'Speed(m/s)_std',
 'energy_y',
 'energy_x',
 'energy_z',
 'entropy_y',
 'entropy_z']

In [0]:
main_columns.append('label')

In [0]:
main_columns

Out[ 27 ]: 
['Speed(m/s)_min',
 'Speed(m/s)_max',
 'Speed(m/s)_mean',
 'Speed(m/s)_std',
 'energy_y',
 'energy_x',
 'energy_z',
 'entropy_y',
 'entropy_z',
 'label']

In [0]:
sensor_data= sensor_data_org[main_columns]

In [0]:
potholes=sensor_data[sensor_data['label']==1]
normal_obs=sensor_data[sensor_data['label']==0] # Create a dataframe for non-pothole observations
print(potholes.count())
print(normal_obs.count())

Speed(m/s)_min 336
Speed(m/s)_max 336
Speed(m/s)_mean 336
Speed(m/s)_std 336
energy_y 336
energy_x 336
energy_z 336
entropy_y 336
entropy_z 336
label 336
dtype: int64
Speed(m/s)_min 6591
Speed(m/s)_max 6591
Speed(m/s)_mean 6591
Speed(m/s)_std 6591
energy_y 6591
energy_x 6591
energy_z 6591
entropy_y 6591
entropy_z 6591
label 6591
dtype: int64

In [0]:
ratio_imb=potholes.count()/normal_obs.count()
#Randomly select datapoints from normal observations to balance dataset
normal_obs_balance=normal_obs.sample(frac=ratio_imb, replace=False, random_state=42)
balanced_sensordata=normal_obs_balance.union(potholes)

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-4285607808499038> in <module> () 
 1 ratio_imb = potholes . count ( ) / normal_obs . count ( ) 
 2 #Randomly select datapoints from normal observations to balance dataset 
 ----> 3 normal_obs_balance = normal_obs . sample ( frac = ratio_imb , replace = False , random_state = 42 ) 
 4 balanced_sensordata = normal_obs_balance . union ( potholes ) 

 /databricks/python/lib/python3.5/site-packages/pandas/core/generic.py in sample (self, n, frac, replace, weights, random_state, axis) 
 2632 raise ValueError ( "Only integers accepted as `n` values" ) 
 2633 elif n is None and frac is not None : 
 -> 2634 n = int ( round ( frac * axis_length ) ) 
 2635 elif n is not None and frac is not None : 
 2636 raise ValueError('Please enter a value for `frac` OR `n`, not '

 /databricks/python/lib/python3.5/site-packages/pandas/core/series.py in wrapper (self) 
 91 return converter ( self . iloc [ 0 ] ) 
 92 raise TypeError("cannot convert the series to "
 ---> 93 "{0}".format(str(converter)))
 94 
 95 return wrapper 

 TypeError : cannot convert the series to <class 'int'>

In [0]:
#check the union is right
balanced_sensordata.where( "label = 0" ).count()

Out[ 5 ]: 353

In [0]:
balanced_sensordata.where( "label = 1" ).count()

Out[ 6 ]: 336

In [0]:
balanced_sensordata_pandas=balanced_sensordata.toPandas() # convert Spark dataframe to pands dataframe

In [0]:
dataset=balanced_sensordata_pandas.sample(frac=1, random_state=42).reset_index(drop=True)

In [0]:
variance_threshold_selector(dataset[main_cols], 0.16)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-2513819918060159> in <module> () 
 ----> 1 variance_threshold_selector ( dataset [ main_cols ] , 0.16 ) 

 NameError : name 'dataset' is not defined

In [0]:
display(selected_features)

In [0]:
dataset[main_cols[:-1]].values.shape

Out[ 12 ]: (689, 38)

In [0]:
# function to separate target and features

def feat_target(data):
  feat= data.drop(['label'], axis=1)
  target=data['label']
  return feat, target

In [0]:
# define grid search function
def grid_search(train, dev, model_config):
  
  #devide in features and target
  X_train, y_train= feat_target(train)
  X_dev, y_dev= feat_target(dev)
  
#   X_train=train.drop(['label'], axis=1)
#   y_train=train['label']
  
#   X_dev=dev.drop(['label'], axis=1)
#   y_dev=dev['label']
  
  #initialize best_model
  ModelRun=namedtuple('Modelrun', ['accuracy', 'precision', 'recall','f1', 'clf', 'name'])
  best_model=ModelRun(accuracy=-1, precision=-1, recall=-1, f1=-1 ,clf=None, name=None)
  
  #define functions
  #comparemodel to current best performing model
  def return_best_model(current_model, contestant_model):
    if contestant_model.accuracy >= current_model.accuracy and contestant_model.precision >= current_model.precision and contestant_model.recall >= current_model.recall:
      return contestant_model
    return current_model
  
  #train model, predict val and evaluate the model
  def train_and_predict(clf, X_train, y_train, X_dev, y_dev):
    probability=True
    clf.fit(X_train, y_train) # fit the model
    y_pred=clf.predict(X_dev) # predict the label of development set
    y_proba=clf.predict_proba(X_dev) #predict the probability of instances predicted in dev set
    accuracy= accuracy_score(y_dev, y_pred)
    precision, recall, f1, support= precision_recall_fscore_support(y_dev, y_pred, average='weighted') # compute accuracy scores 
    return accuracy, precision, recall, f1
  
  # predict using the best model, and get accuracy scores
  def grid_best_model(params, model_class, name, best_model):
    clf=model_class(**params)
    accuracy, precision, recall, f1= train_and_predict(clf, X_train, y_train, X_dev, y_dev)
    best_model=return_best_model(best_model, ModelRun(accuracy=accuracy, precision=precision, recall=recall, f1=f1, clf=clf, name=name))
    return best_model
  # Run grid search
  for i in model_config:
    if len(i.params) >= 1 or i.params is None:
      params_list = list(itertools.product(*i.params.values()))
      for params_values in params_list:
        params = dict(zip(i.params.keys(), params_values))
        best_model = grid_best_model(params, i.model_class, i.name, best_model)
        
  return best_model

In [0]:
#Create train and dev set
def split_train_dev(dataset):
  train_dev = dataset.head(round(len(dataset) * 0.8))  # take the first 80% as train and validation set
  # Splitting
  idx = round(len(train_dev) * 0.8)
  train = train_dev.iloc[:idx]  # take the first 80% as train set
  val = train_dev.iloc[idx:] # take the remaining 20% as validation set
  return train, val

In [0]:
def split_train_test(dataset):
#   train_test = dataset.head(round(len(dataset) * 0.8))  # take the first 80% as train and validation set
  # Splitting
  idx = round(len(dataset)* 0.8)
  train = dataset.iloc[:idx]  # take the first 80% as train set
  test  = dataset.iloc[idx:] # take the remaining 20% as validation set
  return train, test

In [0]:
# col_names = dataset.columns[:-1].tolist()

# features = dataset[col_names]
# scaled_features = StandardScaler().fit(features.values).transform(features.values)
# col_names=dataset.columns[:-1].tolist()
# dataset[col_names] = scaled_features

In [0]:
def tune_model(dataset):
  # Creating a train and validation set
#   train, dev= split_train_dev(dataset)

#   # Scale numeric features
#   col_names = train.columns[:-1].tolist()

#   features = train[col_names]
#   scaled_features = StandardScaler().fit(features.values).transform(features.values)
#   train[col_names] = scaled_features

#   features = dev[col_names]
#   scaled_features = StandardScaler().fit(features.values).transform(features.values)
#   dev[col_names] = scaled_features
  
  
  # Set parameters we want to test
  svc_params=  {'C': [0.5, 0.7, 0.9, 1]
                , 'kernel': ['rbf', 'poly', 'sigmoid', 'linear'],
                'probability':[True]
               }
  
  log_params= {"penalty": ['l1', 'l2']
               ,'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
              }
  
  
  
  xgb_params = {'n_jobs': [48]
                ,'eta': [0.3]
                ,'n_estimators': [3, 10, 30]
                ,'max_depth': [5, 10, 50]
                ,'min_child_weight': [1, 10]
                ,'gamma': [0.1]
                #,'scale_pos_weight': ['auto']
                ,'reg_alpha': [0.1]
               }
  
  rf_params = {'max_depth': [5, 10, 50]
               ,'n_estimators': [3, 10, 30]
               #,'max_features': ['auto', None]
               ,'min_samples_leaf': [2, 5, 10]
               ,'oob_score': [False]
              }
  
  knn_params = {'n_neighbors': [3, 5, 10]
               ,'weights': ['uniform', 'distance']}
  
  adab_params = {'n_estimators': [50, 100]}
  
  nb_params = {}
  
  # Initialise model_config
  ModelConfig = namedtuple('ModelConfig', ['name', 'model_class', 'params'])
  model_config = [ModelConfig(name='SVC', model_class=SVC, params=svc_params),
                  ModelConfig(name='Logistic Regression', model_class=LogisticRegression, params=log_params),
                   ModelConfig(name='XGBoost', model_class=XGBClassifier, params=xgb_params),
                   ModelConfig(name='Random Forest', model_class=RandomForestClassifier, params=rf_params), 
                   ModelConfig(name='KNN', model_class=KNeighborsClassifier, params=knn_params),
                   ModelConfig(name='AdaBoost', model_class=AdaBoostClassifier, params=adab_params),
                   ModelConfig(name='Naive Bayes', model_class=GaussianNB, params=nb_params)
                 ]

  train, dev = split_train_dev(dataset)
  
  # Execute grid search
  
  best_model = grid_search(train, dev, model_config)
  
  
  return best_model.clf, best_model.name, best_model.accuracy, best_model.precision, best_model.recall, best_model.f1

In [0]:
def test_model(dataset, clf):
  if clf == None:
    return -1, -1, -1, -1
  
  train, test =split_train_test(dataset)
 
  
  
  # divide in X and y
  X_train, y_train = feat_target(train)
  X_test, y_test   = feat_target(test)

  def train_and_predict(clf, X_train, y_train, X_test, y_test):
    clf.fit(X_train, y_train) # fit the model
    y_prob=clf.predict_proba(X_test)
    y_pred=clf.predict(X_test) # predict the label of development set
    test_accuracy= accuracy_score(y_test, y_pred)
    test_precision, test_recall, test_f1, test_support= precision_recall_fscore_support(y_test, y_pred, average='weighted') # compute accuracy scores 
    return test_accuracy, test_precision, test_recall, test_f1
  
  
  # Train classifier and output scores
  test_accuracy, test_precision, test_recall,test_f1 = train_and_predict(clf, X_train, y_train, X_test, y_test)
  
  return test_accuracy, test_precision, test_recall, test_f1


In [0]:
#mlflow.set_experiment("/Users/zeinab.bakhtiarinoodeh@tomtom.com/pothole_detection_new")

In [0]:
# cntx = dbutils.entry_point.getDbutils().notebook().getContext()
# api_token = cntx.apiToken().get()
# api_url = cntx.apiUrl().get()


def create_model(dataset):  
  # Collect a tuned model for this user, if number of classes is < 2, return False
  clf, name, val_accuracy, val_precision, val_recall, val_f1 = tune_model(dataset)

  # Test performance on the test set (with tuned model)
  test_accuracy, test_precision, test_recall, test_f1 = test_model(dataset, clf)
  
#   os.environ["DATABRICKS_TOKEN"] = api_token
#   os.environ["DATABRICKS_HOST"] = api_url
  
  
  
  with mlflow.start_run() as run:
#     mlflow.set_tag("project", "Pothole detection")
    if clf:
      mlflow.log_param("model_name", name)
      for param_key, param_value in clf.get_params().items():
        mlflow.log_param("model_" + param_key, param_value)
      mlflow.sklearn.log_model(clf, "model")
   

    mlflow.log_metric("val_accuracy", val_accuracy)
    mlflow.log_metric("test_accuracy", test_accuracy)
    mlflow.log_metric("val_precision", val_precision)
    mlflow.log_metric("test_precision", test_precision)
    mlflow.log_metric("val_recall", val_recall)
    mlflow.log_metric("test_recall", test_recall)
    mlflow.log_metric("val_f1", val_f1)
    mlflow.log_metric("test_f1", test_f1)

  
  # Store results in dataframe
  result = pd.DataFrame([[name] + [val_accuracy] + [test_accuracy] + [val_precision] + [test_precision] + [val_recall] + [test_recall]+ [val_f1]+[test_f1]], 
                        columns=['classifier', 'val_accuracy', 'test_accuracy', 'val_precision', 'test_precision', 'val_recall', 'test_recall', 'val_f1', 'test_f1'])
 
  return result, clf

In [0]:
result, clf= create_model(selected_features)
